In [1]:
import requests
import pandas as pd
import numpy as np

# soil properties
- for one coordinaters
- for one property

 Varaibles
 - Physical soil properties
     - BLDFIE: Bulk Density ('kg/cubic_m')
     - CLYPPT: Caly Content (percent)
     - CRFVOL: Coarse Fragements Volumetric (%) (percent)
     - SLTPPT: Slit Content (percent)
     - SNDPPT: Sand Content (percent)
 - Chemical Soil Properties
     - CECSOL: Cation Excahnge capacit of soil (cmol / kg)
     - ORCDRC: Soil organic carbon content (g / kg)
     - PHIHOX: Soil pH * 10 in H20 (Index*10)
     - PHIKCL: Soil pH * 10 in KCL (Index*10)

## first we keep 2 decimals for the lat and long 

In [59]:
#soilProperties =['BLDFIE','CLYPPT','CRFVOL','SLTPPT', 'SNDPPT','CECSOL','ORCDRC','PHIHOX','PHIKCL']
lat = round(-77.85,2)
long = round( 166.76,2)

# lat = round(29.1,2)
# long = round(-81.633,2)
webPath = "https://rest.soilgrids.org/query?lon="
property = 'BLDFIE'
resp = requests.get(webPath + str(round(long,2)) +
        "&lat=" + str(round(lat,2)) + "&attributes=%s"%str(property))

## no this property existed

In [65]:
# if there is no this property existed for the coordinates
# then keep one decimals
if (property not in pd.DataFrame(resp.json()).index) == True:
    print ('there is no this property existed for the coordinates')
    resp = requests.get(webPath + str(round(long, 1)) +
                        "&lat=" + str(round(lat, 1)) + "&attributes=%s" % str(property))
    
    if resp.json()['properties']['soilmask'] == 'nodata':
        print('yes')


    

there is no this property existed for the coordinates
yes


In [52]:
pd.DataFrame(resp.json()).index

Index(['type', 'coordinates', 'properties', 'soilmask', 'BLDFIE',
       'depthCodesMeters', 'publication_date'],
      dtype='object')

In [49]:
'properties' in resp.json()

True

In [34]:
resp =  requests.get(webPath + str(round(long, 2)) +
                        "&lat=" + str(round(lat, 2)) + "&attributes=%s" % str(property))

In [50]:
resp.json().keys()

dict_keys(['type', 'geometry', 'crs', 'timestamp', 'raw', 'properties'])

In [57]:
resp.json()['properties']['soilmask'] == 

'soil'

In [60]:
resp.json().keys()

dict_keys(['type', 'geometry', 'crs', 'timestamp', 'raw', 'properties'])

In [63]:
resp.json()['properties']['soilmask'] == 'nodata'

True

In [21]:
# get the data with one property
sigData = pd.DataFrame(resp.json()).loc[property]

KeyError: 'BLDFIE'

In [5]:
# only get useful column - 'properties'
# transform the dictionay 'properties' as dataframe
transData = pd.DataFrame.from_dict(sigData.properties)

In [6]:
# if we still keep two decimals
# none values for this specific coordinates
if all(v is None for v in transData.M) == True:
    resp = requests.get(webPath + str(round(long, 1)) +
                        "&lat=" + str(round(lat, 1)) +
                        "&attributes=%s" % str(property))
    sigData = pd.DataFrame(resp.json()).loc[property]
    transData = pd.DataFrame.from_dict(sigData.properties)

In [7]:
#  transpose
preparedSigData = transData.T.tail(1).add_prefix('%s_'%property).reset_index(drop=True)
preparedSigData

,BLDFIE_sl1,BLDFIE_sl2,BLDFIE_sl3,BLDFIE_sl4,BLDFIE_sl5,BLDFIE_sl6,BLDFIE_sl7
0,1214,1319,1391,1500,1568,1593,1584


# Top three soil types

In [8]:
classProperties = ['TAXNWRB', 'TAXOUSDA']
property = 'TAXNWRB'

In [9]:
resp = requests.get(webPath + str(round(long,2)) +
                    "&lat=" + str(round(lat,2)) + "&attributes=%s"%str(property))

In [10]:
# no this property existed for the coordinates, then round 1
if (property not in pd.DataFrame(resp.json()).index) == True:
    resp = requests.get(webPath + str(round(long, 1)) +
                        "&lat=" + str(round(lat, 1)) + "&attributes=%s" % str(property))

In [11]:
sigData = pd.DataFrame(resp.json())
classDict = sigData.loc[property].properties
classDict

{'Acric Ferralsols': 0,
 'Acric Plinthosols': 0,
 'Albic Arenosols': 0,
 'Albic Luvisols': 0,
 'Alic Nitisols': 1,
 'Aluandic Andosols': 1,
 'Aric Regosols': 1,
 'Calcaric Regosols': 1,
 'Calcic Chernozems': 0,
 'Calcic Gleysols': 0,
 'Calcic Gypsisols': 0,
 'Calcic Histosols': 0,
 'Calcic Kastanozems': 0,
 'Calcic Luvisols': 0,
 'Calcic Solonetz': 0,
 'Calcic Vertisols': 3,
 'Cryic Histosols': 0,
 'Cutanic Alisols': 0,
 'Endogleyic Cambisols': 0,
 'Endogleyic Planosols': 0,
 'Ferralic Arenosols': 0,
 'Ferralic Cambisols': 0,
 'Fibric Histosols': 0,
 'Gleyic Luvisols': 1,
 'Gleyic Podzols': 4,
 'Gleyic Solonetz': 1,
 'Gypsic Solonchaks': 0,
 'Haplic Acrisols': 15,
 'Haplic Acrisols (Alumic)': 0,
 'Haplic Acrisols (Ferric)': 1,
 'Haplic Acrisols (Humic)': 0,
 'Haplic Albeluvisols': 1,
 'Haplic Alisols': 4,
 'Haplic Andosols': 0,
 'Haplic Arenosols': 17,
 'Haplic Arenosols (Calcaric)': 0,
 'Haplic Calcisols': 0,
 'Haplic Calcisols (Sodic)': 0,
 'Haplic Cambisols': 2,
 'Haplic Cambisols (

In [12]:
# none values of property for this specific coordinates is higher than 95%, then round1
if sum(v is None for v in classDict.values())/len(classDict) > 0.95:
    print ("round 2: none values of property for this specific coordinates")
    resp = requests.get(webPath + str(round(long, 1)) +
                        "&lat=" + str(round(lat, 1)) + "&attributes=%s" % str(property))
    sigData = pd.DataFrame(resp.json())
    classDict = sigData.loc[property].properties

    # none value for round 1 is still higher than 95%, then imputation with np.nan
    if sum(v is None for v in classDict.values())/len(classDict) > 0.95:
        print ("round 1:none values of property for this specific coordinates")
        mostProbClassDict = {'%s_Class1' % property: np.nan, '%s_Class2' % property: np.nan,
                             '%s_Class3' % property: np.nan}
        sigClassData = pd.DataFrame(mostProbClassDict, index=[0])
        soilClassData = pd.concat([soilClassData, sigClassData], axis=1)   

In [13]:
sortedDict = sorted(classDict, key=lambda k: classDict[k], reverse=True)

# get top three classes with high probability
mostProbClassDict = {'%s_Class1' % property: sortedDict[0], '%s_Class2' % property: sortedDict[1],
                     '%s_Class3' % property: sortedDict[2]}

sigClassData = pd.DataFrame(mostProbClassDict, index=[0])

In [14]:
sigClassData.head(2)

,TAXNWRB_Class1,TAXNWRB_Class2,TAXNWRB_Class3
0,Haplic Arenosols,Haplic Acrisols,Haplic Luvisols
